<a href="https://colab.research.google.com/github/MuhammadAbuSufian/colab/blob/main/I2IQA_ImageReward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
!pip install image-reward
#!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install git+https://github.com/openai/CLIP.git

import pandas as pd
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests
from io import BytesIO
import os
import ImageReward as RM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab/I2IQA/I2IQA_annotation.csv')
data = raw_data

Mounted at /content/drive


In [25]:
# Initialize an empty list to store results
results = []
expected_results = []

model = RM.load("ImageReward-v1.0")

# Iterate over each row in the dataframe
for index, row in data.iterrows():
    image_name = row['Generated_image']
    prompt = row['Text_prompt']
    human_feedback_score = row['MOS_a']


    # Construct the path to the image file
    image_path = os.path.join('/content/drive/MyDrive/Colab/I2IQA/Generated_image/All/Unziped', image_name)

    # Load the image
    try:
        with Image.open(image_path) as img:
            reward = model.score(prompt, img)
            results.append(reward)
            expected_results.append(human_feedback_score)
    except IOError:
        print(f"Failed to load image {image_name}")

# Convert results to a DataFrame for easy viewing or further processing

print(results)
print(expected_results)

load checkpoint from /root/.cache/ImageReward/ImageReward.pt
checkpoint loaded


KeyboardInterrupt: 

In [22]:

print(results)
print(expected_results)

[1.3911728858947754, 0.8891820311546326, 1.3083109855651855, 1.31610906124115, 0.0630575492978096, 1.3647499084472656, 1.747051477432251, 1.960457682609558, 1.7967610359191895, 1.3022048473358154, -1.1388193368911743, 1.8454707860946655, 0.8135002851486206, 0.4765174388885498, -0.9789621233940125, 0.6215441226959229, 0.6533812284469604, 0.9275092482566833, 1.0010350942611694, 1.512823224067688, 1.5214790105819702, 1.8713712692260742, 1.825225591659546, 1.8602931499481201, 0.463027685880661, 0.29587000608444214, 0.5310595035552979, -0.48758405447006226, 1.9076040983200073, 1.849937915802002, 1.9043546915054321, 1.893631935119629, 0.5690156817436218, 0.5151453614234924, 1.6070574522018433, -0.7690957188606262, 1.499434471130371, 1.2856855392456055, 1.518693208694458, 1.1935505867004395, 1.4729431867599487, 1.2628662586212158, 1.032568097114563, 1.4732317924499512, 1.0454704761505127, 1.2382310628890991, 1.4705476760864258, 1.020329236984253, 1.0261662006378174, 0.041764844208955765, 1.26

In [24]:
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import re  # Import regular expressions

# Helper function to clean and convert data
def clean_and_convert(data):
    cleaned_data = []
    for item in data:
        try:
            # Ensure item is a string and remove non-numeric characters
            cleaned_item = re.sub(r'[^0-9\.]', '', str(item))
            cleaned_data.append(float(cleaned_item))
        except ValueError:
            # Handle the case where conversion is not possible
            print(f"Skipping item {item} due to conversion error.")
    return np.array(cleaned_data, dtype=float)


# Clean and convert strings to floats
subjective_scores = clean_and_convert(expected_results)
objective_scores = np.array(results, dtype=float)

print("Sizes:", subjective_scores.size, objective_scores.size)

# Calculate PLCC
plcc, _ = pearsonr(subjective_scores, objective_scores)

# Calculate SROCC
srocc, _ = spearmanr(subjective_scores, objective_scores)

# Calculate KROCC
krocc, _ = kendalltau(subjective_scores, objective_scores)

print("PLCC:", plcc)
print("SROCC:", srocc)
print("KROCC:", krocc)


Sizes: 1600 1600
PLCC: 0.13274575767482133
SROCC: 0.11831481078986403
KROCC: 0.07956194076378992
